In [1]:
import os
import glob
import random
import pandas as pd
import datetime
import tiktoken
from collections import Counter, defaultdict
from IPython.display import display, Markdown

# user DF에서 History column과 Train column을 합쳐서 history로 사용
file_name = "MIND_LLM.tsv"
user = pd.read_csv(f'../../../data/LLM/{file_name}', sep='\t', names=['User', 'History', 'Question'])

NNR_news = pd.read_csv('../../../data/baseline/MIND_news.tsv', sep='\t', names=['News ID','Category','SubCategory','Title','Abstract','URL','Title Entities','Abstract Entities'])
NNR_news = NNR_news.drop(columns=['URL', 'Title Entities', 'Abstract Entities'])

display(Markdown("### 1. user<hr/>"))
# display(user[user['User']==1]['Question'][0])
display(user)
display(Markdown("### 2. NNR_news<hr/>"))
display(NNR_news)

### 1. user<hr/>

,User,History,Question
0,1,N58584 N12900 N6233 N51706 N18777 N593 N16793 ...,N31978 N24207 N58271 N14184 N36789
1,1,N58584 N12900 N6233 N51706 N18777 N593 N16793 ...,N57809 N50675 N61404 N32544 N47602
2,1,N58584 N12900 N6233 N51706 N18777 N593 N16793 ...,N28213 N29801 N60992 N19592 N62128
3,2,N56586 N3046 N4209 N22293 N40545 N44495 N38659...,N22061 N4247 N61648 N27560 N22407
4,2,N56586 N3046 N4209 N22293 N40545 N44495 N38659...,N3128 N58295 N17965 N34099 N55186
...,...,...,...
187613,37639,N22185 N56586 N3500 N61826 N10629 N9824 N2745 ...,N8353 N59088 N33885 N50490 N17882
187614,37639,N22185 N56586 N3500 N61826 N10629 N9824 N2745 ...,N63058 N39151 N44183 N26224 N23615
187615,37639,N22185 N56586 N3500 N61826 N10629 N9824 N2745 ...,N61828 N41717 N8353 N48783 N46098
187616,37639,N22185 N56586 N3500 N61826 N10629 N9824 N2745 ...,N17228 N4156 N51220 N12330 N54869


### 2. NNR_news<hr/>

,News ID,Category,SubCategory,Title,Abstract
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."
...,...,...,...,...,...
51277,N16909,weather,weathertopstories,"Adapting, Learning And Soul Searching: Reflect...",Woolsey Fire Anniversary: A community is forev...
51278,N47585,lifestyle,lifestylefamily,Family says 13-year-old Broadway star died fro...,NaN
51279,N7482,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b..."
51280,N34418,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ..."


In [2]:
user[user['User']==1]['History'][0]

'N58584 N12900 N6233 N51706 N18777 N593 N16793 N846 N18285 N46846 N53436'

In [4]:
user_num = 2

li = user[user['User']==user_num]['History'].iloc[0].split(' ')
print('[History]')
for i, num in enumerate(li):
    print(f"{i+1}. ",NNR_news[NNR_news['News ID']==num]['Title'].iloc[0])


print('\n[Question]')
li = user[user['User']==user_num]['Question'].str.split().str[0].tolist()
for i, num in enumerate(li):
    print(f"{i+1}. ",NNR_news[NNR_news['News ID']==num]['Title'].iloc[0])

[History]
1.  Prince William and Kate Middleton arrive in Pakistan for royal tour
2.  See all the best photos of Prince William and Duchess Kate's royal tour of Pakistan
3.  6 Signs You're Not Getting Enough Potassium
4.  Man sets pet lion on electrician who tried to collect pay, police say
5.  12 things that could save us money if we weren't lazy
6.  Unprecedented movement detected on California earthquake fault capable of 8.0 temblor
7.  France says foiled September 11-inspired attack
8.  These 4 words are the key to becoming rich
9.  7 Surprising Features That Boost Your Home Value
10.  30 Funny Dog Photos You Need to See
11.  This is the best exercise to pump up aging muscles
12.  Bag Explodes While Being Loaded On Volaris Flight At Midway Airport
13.  Reduce Stomach Bloating: 9 Daily Habits That Flatten Your Belly
14.  Duchess Meghan Returns to Public Eye After Africa Tour Documentary Airs
15.  Body of missing Alabama girl found; 2 being charged
16.  Why Healthy People Faint and H